In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

BASE_URL = "https://www.ourlads.com"
DEPTHCHARTS_INDEX = BASE_URL + "/nfldepthcharts/depthcharts.aspx"

# Target positions (including variants for WR)
TARGET_POS = {"QB", "WR", "RB", "TE", "LWR", "RWR", "SWR"}

def get_team_links():
    resp = requests.get(DEPTHCHARTS_INDEX, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/nfldepthcharts/depthchart/"):
            full = BASE_URL + href
            if full not in links:
                links.append(full)
    return links

def parse_team_depthchart(team_url):
    resp = requests.get(team_url, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Derive team abbreviation
    team_abbrev = None
    depwrapper = soup.find("div", id="ctl00_phContent_DepWrapper")
    if depwrapper:
        for c in depwrapper.get("class", []):
            if c.startswith("dt-"):
                team_abbrev = c[3:]
                break
    if not team_abbrev:
        team_abbrev = team_url.rstrip("/").split("/")[-1].upper()

    records = []
    # Locate the table
    table = None
    if depwrapper:
        table = depwrapper.find("table", class_="table-bordered")
    if table is None:
        table = soup.find("table", class_="table-bordered")
    if table is None:
        return records

    tbody = table.find("tbody")
    if tbody is None:
        tbody = table.find("tbody", id="ctl00_phContent_dcTBody")
    if tbody is None:
        return records

    for tr in tbody.find_all("tr"):
        tds = tr.find_all("td")
        if len(tds) < 2:
            continue

        pos_raw = tds[0].get_text(strip=True)
        pos = pos_raw.strip()
        if pos in ("LWR", "RWR", "SWR"):
            pos_norm = "WR"
        else:
            pos_norm = pos

        if pos_norm not in TARGET_POS:
            continue

        # Iterate over player slots:
        # The “player slots” are at tds indices 2, 4, 6, 8, ... ; their corresponding “No.” are at 1, 3, 5, 7, ...
        # We'll index pairs: (num_idx, player_idx) = (1,2), (3,4), (5,6), (7,8), ...
        # Tier is determined by which “player position slot” (1st -> tier 1, 2nd -> tier 2, etc.)
        tier = 1
        for num_idx in range(1, len(tds), 2):
            player_idx = num_idx + 1
            if player_idx >= len(tds):
                break
            a = tds[player_idx].find("a")
            if a and a.get_text(strip=True):
                player_text = a.get_text(strip=True)

                # Remove trailing depth chart key like "SF24", "CF22", "SF21", etc.
                player_text = re.sub(
                  r"\s+(?:[A-Z]{2}\d{2}|\d{2}/\d|[A-Z]{1,2}/[A-Za-z]{2,3})$",
                  "",
                  player_text,
                  flags=re.IGNORECASE,
                )

                # Normalize capitalization (proper case)
                player_text = player_text.title()

                player_clean = player_text.strip()

                records.append((player_clean, team_abbrev, pos_norm, tier))


            tier += 1

    return records

def scrape_all(save_csv_path="nfl_players_with_tiers_2025.csv"):
    team_links = get_team_links()
    print(f"Found {len(team_links)} team pages.")
    all_records = []
    for link in team_links:
        try:
            recs = parse_team_depthchart(link)
            print(f"  {link} → {len(recs)} records")
            all_records.extend(recs)
        except Exception as e:
            print(f"Error parsing {link}: {e}")
        time.sleep(1)

    # Deduplicate (optionally)
    seen = set()
    deduped = []
    for rec in all_records:
        if rec not in seen:
            seen.add(rec)
            deduped.append(rec)

    with open(save_csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Player", "Team", "Position", "Tier"])
        for rec in deduped:
            writer.writerow(rec)
    print(f"Wrote {len(deduped)} unique records to {save_csv_path}")

if __name__ == "__main__":
    scrape_all()


Found 32 team pages.
  https://www.ourlads.com/nfldepthcharts/depthchart/BUF → 14 records
  https://www.ourlads.com/nfldepthcharts/depthchart/MIA → 15 records
  https://www.ourlads.com/nfldepthcharts/depthchart/NE → 13 records
  https://www.ourlads.com/nfldepthcharts/depthchart/NYJ → 15 records
  https://www.ourlads.com/nfldepthcharts/depthchart/BAL → 12 records
  https://www.ourlads.com/nfldepthcharts/depthchart/CIN → 15 records
  https://www.ourlads.com/nfldepthcharts/depthchart/CLE → 14 records
  https://www.ourlads.com/nfldepthcharts/depthchart/PIT → 11 records
  https://www.ourlads.com/nfldepthcharts/depthchart/HOU → 16 records
  https://www.ourlads.com/nfldepthcharts/depthchart/IND → 15 records
  https://www.ourlads.com/nfldepthcharts/depthchart/JAX → 14 records
  https://www.ourlads.com/nfldepthcharts/depthchart/TEN → 14 records
  https://www.ourlads.com/nfldepthcharts/depthchart/DEN → 15 records
  https://www.ourlads.com/nfldepthcharts/depthchart/KC → 17 records
  https://www.o